In [ ]:
from dotenv import load_dotenv
import os
import wandb

load_dotenv()

wandb_api_key = os.getenv("wand_api_key")

!wandb login $wandb_api_key
wandb.login()

run = wandb.init(project='huggingface')
artifact = run.use_artifact('fine-tuned-model:latest', type="model")
artifact_dir = artifact.download()

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.wandb: WARNING Calling wandb.login() after wandb.init() has no effect.

Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\Scripts\wandb.exe\__main__.py", line 7, in <module>
  File "C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages

In [15]:
import torch
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForSequenceClassification
from datasets import Dataset

model_name = "ProsusAI/finbert"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

model_path = f"{artifact_dir}/model.pth"
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

model.eval()


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [16]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained(model_name)

text = "The stock market is performing exceptionally well today."

inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

print(predictions)


tensor([[6.0553e-04, 3.0274e-03, 9.9637e-01]])


In [17]:
import requests
import json
from datetime import datetime, timedelta


class NewsRetrievalAgent:
    def __init__(self, api_key):
        self.api_key = api_key
        self.base_url = "https://newsapi.org/v2/everything?q=NSE"

    def fetch_news(self, query="IPO OR initial public offering", days=7, sources=None):
        date_from = (datetime.now() - timedelta(days=days)).strftime('%Y-%m-%d')
        params = {
            "q": query,
            "from": date_from,
            "language": "en",
            "sortBy": "publishedAt",
            "apiKey": self.api_key
        }
        if sources:
            params["sources"] = sources
        
        response = requests.get(self.base_url, params=params)
        if response.status_code == 200:
            data = response.json()
            return data.get("articles", [])
        else:
            print(f"Error fetching news: {response.status_code}")
            return []

    def summarize_news(self, articles, top_n=5):
        summaries = []
        for article in articles[:top_n]:
            summaries.append({
                "title": article["title"],
                "source": article["source"]["name"],
                "published_at": article["publishedAt"],
                "url": article["url"]
            })
        return summaries

news_api_key = os.getenv("news_api_key")
news_agent = NewsRetrievalAgent(news_api_key)
news_articles = news_agent.fetch_news()
summaries = news_agent.summarize_news(news_articles)

for news in summaries:
    print(news)

{'title': 'At multi-year highs: These 3 stocks witness 5-year swing high breakouts', 'source': 'Economictimes.com', 'published_at': '2025-03-25T03:12:52Z', 'url': 'https://m.economictimes.com/markets/stocks/news/at-multi-year-highs-these-3-stocks-witness-5-year-swing-high-breakouts/rising-high/slideshow/119453718.cms'}
{'title': 'ATC Energies System IPO to open for subscription today. Check issue price, GMP, other details', 'source': 'The Times of India', 'published_at': '2025-03-25T02:14:41Z', 'url': 'https://economictimes.indiatimes.com/markets/ipos/fpos/atc-energies-system-ipo-to-open-for-subscription-today-check-issue-price-gmp-other-details/articleshow/119451851.cms'}
{'title': 'Shri Ahimsa Naturals IPO to open for subscription today. Check issue price, GMP, other details', 'source': 'The Times of India', 'published_at': '2025-03-25T02:12:09Z', 'url': 'https://economictimes.indiatimes.com/markets/ipos/fpos/shri-ahimsa-naturals-ipo-to-open-for-subscription-today-check-issue-price-g

In [18]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

model_name = "ProsusAI/finbert"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

model_path = f"{artifact_dir}/model.pth"
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

def analyze_sentiment(news_articles):
    results = []
    for article in news_articles:
        text = article["title"] + " " + article.get("description", "")
        inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
        
        with torch.no_grad():
            outputs = model(**inputs)
        
        logits = outputs.logits
        sentiment = torch.softmax(logits, dim=1).tolist()[0]
        article["sentiment"] = {
            "negative": sentiment[0],
            "neutral": sentiment[1],
            "positive": sentiment[2]
        }
        results.append(article)

    return results

news_articles = news_agent.summarize_news(news_articles)
news_with_sentiment = analyze_sentiment(news_articles)

for news in news_with_sentiment:
    print(news["title"], news["sentiment"])


At multi-year highs: These 3 stocks witness 5-year swing high breakouts {'negative': 0.0009772450430318713, 'neutral': 0.0020051468163728714, 'positive': 0.9970176219940186}
ATC Energies System IPO to open for subscription today. Check issue price, GMP, other details {'negative': 0.001582072232849896, 'neutral': 0.9787472486495972, 'positive': 0.019670572131872177}
Shri Ahimsa Naturals IPO to open for subscription today. Check issue price, GMP, other details {'negative': 0.0015661967918276787, 'neutral': 0.9793745279312134, 'positive': 0.01905926875770092}
Share Market Highlights: Sensex, Nifty hold steady; IT stocks shine, midcaps retreat {'negative': 0.0014977715909481049, 'neutral': 0.12336458265781403, 'positive': 0.8751376271247864}
Bulls in beast mode, clean up D-Street losses {'negative': 0.0012552207335829735, 'neutral': 0.9954403638839722, 'positive': 0.003304416546598077}


In [ ]:
import requests
import csv
import os
from datetime import datetime
from transformers import BertTokenizer, BertForSequenceClassification
import torch

model_name = "ProsusAI/finbert"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

model_path = f"{artifact_dir}/model.pth"
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

NEWS_URL = "https://newsapi.org/v2/everything"
QUERY = 'NSE OR "National Stock Exchange of India"'
LANGUAGE = "en"

def fetch_nse_news():
    params = {
        "q": QUERY,
        "language": LANGUAGE,
        "apiKey": news_api_key
    }
    response = requests.get(NEWS_URL, params=params)
    if response.status_code == 200:
        return response.json().get("articles", [])
    else:
        print(f"Error fetching news: {response.status_code}")
        return []

def analyze_sentiment(news_articles):
    results = []
    for article in news_articles:
        text = article["title"] + " " + article.get("description", "")
        inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
        
        with torch.no_grad():
            outputs = model(**inputs)

        logits = outputs.logits
        sentiment = torch.softmax(logits, dim=1).tolist()[0]
        
        article["sentiment"] = {
            "negative": sentiment[0],
            "neutral": sentiment[1],
            "positive": sentiment[2]
        }
        results.append(article)

    return results

def save_to_csv(news_with_sentiment, filename="nse_news_sentiment.csv"):
    file_exists = os.path.exists(filename)
    with open(filename, mode="a", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        
        if not file_exists:
            writer.writerow(["Date", "Title", "Source", "URL", "Positive", "Neutral", "Negative"])

        for article in news_with_sentiment:
            writer.writerow([
                datetime.now().strftime("%Y-%m-%d"),
                article["title"],
                article["source"]["name"],
                article["url"],
                article["sentiment"]["positive"],
                article["sentiment"]["neutral"],
                article["sentiment"]["negative"]
            ])
    print(f"Saved {len(news_with_sentiment)} articles to {filename}")

if __name__ == "__main__":
    news_articles = fetch_nse_news()
    if news_articles:
        news_with_sentiment = analyze_sentiment(news_articles)
        save_to_csv(news_with_sentiment)
    else:
        print("No news articles retrieved.")


Saved 100 articles to nse_news_sentiment.csv


In [21]:
import requests
import csv
import os
from datetime import datetime
from transformers import BertTokenizer, BertForSequenceClassification
import torch

model_name = "ProsusAI/finbert"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

model_path = f"{artifact_dir}/model.pth"
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

NEWS_URL = "https://newsapi.org/v2/everything"
QUERY = 'BSE OR "Bombay Stock Exchange of India"'
LANGUAGE = "en"

def fetch_nse_news():
    params = {
        "q": QUERY,
        "language": LANGUAGE,
        "apiKey": news_api_key
    }
    response = requests.get(NEWS_URL, params=params)
    if response.status_code == 200:
        return response.json().get("articles", [])
    else:
        print(f"Error fetching news: {response.status_code}")
        return []

def analyze_sentiment(news_articles):
    results = []
    for article in news_articles:
        text = article["title"] + " " + article.get("description", "")
        inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
        
        with torch.no_grad():
            outputs = model(**inputs)

        logits = outputs.logits
        sentiment = torch.softmax(logits, dim=1).tolist()[0]
        
        article["sentiment"] = {
            "negative": sentiment[0],
            "neutral": sentiment[1],
            "positive": sentiment[2]
        }
        results.append(article)

    return results

def save_to_csv(news_with_sentiment, filename="bse_news_sentiment.csv"):
    file_exists = os.path.exists(filename)
    with open(filename, mode="a", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        
        if not file_exists:
            writer.writerow(["Date", "Title", "Source", "URL", "Positive", "Neutral", "Negative"])

        for article in news_with_sentiment:
            writer.writerow([
                datetime.now().strftime("%Y-%m-%d"),
                article["title"],
                article["source"]["name"],
                article["url"],
                article["sentiment"]["positive"],
                article["sentiment"]["neutral"],
                article["sentiment"]["negative"]
            ])
    print(f"Saved {len(news_with_sentiment)} articles to {filename}")

if __name__ == "__main__":
    news_articles = fetch_nse_news()
    if news_articles:
        news_with_sentiment = analyze_sentiment(news_articles)
        save_to_csv(news_with_sentiment)
    else:
        print("No news articles retrieved.")


Saved 100 articles to bse_news_sentiment.csv
